In [ ]:
import sys
import bookutils
import ipywidgets as widgets

In [ ]:
from Tracer import Tracer

In [ ]:
class State():
    def __init__(self,lineno,code,changedVars,callStack):
        self.lineno = lineno
        self.code = code
        self.changedVars = changedVars
        self.callStack = callStack

    def __str__(self):
        return str(self.lineno) + '   '+ repr(self.code) + '   '+ repr(self.changedVars)

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
class Tracer(object):
    def __init__(self, file=sys.stdout):
        """Trace a block of code, sending logs to file (default: stdout)"""
        self.original_trace_function = None
        self.file = file
        self.log_dict =[]
        pass

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        # print(*objects, sep=sep, end=end, file=self.file, flush=True)
        # self.logger = self.logger + "\n" + " ".join(objects)

        current_state = State(objects[1],objects[2],objects[3],None)
        self.log_dict.append(current_state)


    def traceit(self, frame, event, arg):
        """Tracing function. To be overridden in subclasses."""
        self.log(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)

    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own __exit__() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at begin of `with` block. Turn tracing off."""
        sys.settrace(self.original_trace_function)
        self.get_log()

    def get_log(self):
        index = 0
        for log in self.log_dict:
            print('Log Entry: ' + str(index))
            index = index + 1
            print(log)

In [ ]:
play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
tester = widgets.HBox([play, slider])

def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))

top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

top_left_text = widgets.IntText(description='Top left', layout=widgets.Layout(width='auto', height='auto'))
top_right_text = widgets.IntText(description='Top right', layout=widgets.Layout(width='auto', height='auto'))
bottom_left_slider = widgets.IntSlider(description='Bottom left', layout=widgets.Layout(width='auto', height='auto'))
bottom_right_slider = widgets.IntSlider(description='Bottom right', layout=widgets.Layout(width='auto', height='auto'))

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

widgets.AppLayout(header=tester,
          center=center_button,
          right_sidebar=right_button,
          pane_widths=[2, 0.5, 1],
          pane_heights=[1, 17, 1])


In [ ]:
with Tracer():
    remove_html_markup("<b>bold</b>")


